## 5.3 버킷 집계
### 5.3.1 범위 집계
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "bytes_range": {
      "range": {
        "field": "bytes",
        "ranges": [
          {
            "from": 1000,
            "to": 2000
          }
        ]
      }
    }
  }
}
<br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "bytes_range": {
      "range": {
        "field": "bytes",
        "ranges": [
          {
            "key": "small",
            "to": 1000
          },
          {
            "key": "medium",
            "from": 1000,
            "to": 2000
          },
          {
            "key": "large",
            "from": 2000,
            "to": 3000
          }
        ]
      }
    }
  }
}

### 5.3.2 날짜 범위 집계
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "request count with date range": {
      "date_range": {
        "field": "timestamp",
        "ranges": [
          {
            "from": "2015-01-04T05:14:00.000Z",
            "to": "2015-01-04T05:16:00.000Z"
          }
        ]
      }
    }
  }
}

### 5.3.3 히스토그램 집계
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "bytes_histogram": {
      "histogram": {
        "field": "bytes",
        "interval": 10000
      }
    }
  }
}
<br>
- min_doc_count 통해서 최소 문서 단위 수 지정 <br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "bytes_histogram": {
      "histogram": {
        "field": "bytes",
        "interval": 10000,
        "min_doc_count": 1
      }
    }
  }
}

### 5.3.4 날짜 히스토그램 집계
- 분 단위 로그 분류 <br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "daily_request_count": {
      "date_histogram": {
        "field": "timestamp",
        "interval": "minute"
      }
    }
  }
}
- 반환하는 날짜 포맷 변경 (단, 인지 가능한 포맷에 한정) <br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "daily_request_count": {
      "date_histogram": {
        "field": "timestamp",
        "interval": "day",
        "format": "yyyy-MM-dd"
      }
    }
  }
}
- time_zone : 다른 나라 시간 기준으로 변경 <br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "daily_request_count": {
      "date_histogram": {
        "field": "timestamp",
        "interval": "day",
        "time_zone": "+09:00"
      }
    }
  }
}

- offset : time_zone과 비슷한 효과, 반환하는 날짜 값에 +h time을 더함 <br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "daily_request_count": {
      "date_histogram": {
        "field": "timestamp",
        "interval": "day",
        "offset": "+3h"
      }
    }
  }
}

### 5.3.5 텀즈 집계
- 각 샤드별 상위 10개 (default)에 대한 terms 집계 (sum_other_doc_count: 집계 되지 않은 문서 수) <br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "request_count_by_country": {
      "terms": {
        "field": "geoip.country_name.keyword"
      }
    }
  }
}

- 샤드별 집계할 size 지정 (클수록 정확하지만 비용은 증가)
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "request_count_by_country": {
      "terms": {
        "field": "geoip.country_name.keyword",
         "size": 100
      }
    }
  }
}

## 5.4 파이프라인 집계
### 5.4.1 형제 집계
- 평균/최대/최소/합계/통계/확장통계/백분위수/이동평균 (버킷) 집계 <br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "histo" : {
      "date_histogram": {
        "field": "timestamp",
        "interval": "minute"
      },
      "aggs": {
        "bytes_sum": {
          "sum": {
            "field": "bytes"
          }
        }
      }
    },
    "max_bytes": {
      "max_bucket": {
        "buckets_path": "histo>bytes_sum"
      }
    }
  }
}

### 5.4.2 부모 집계
- 파생/누적/버킷스크립트/버킷셀럭터/시계열차분 집계 <br>
- 파생 집계 <br>
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "histo" : {
      "date_histogram": {
        "field": "timestamp",
        "interval": "day"
      },
      "aggs": {
        "bytes_sum": {
          "sum": {
            "field": "bytes"
          }
        },
        "sum_deriv": {
          "derivative": {
            "buckets_path": "bytes_sum"
          }
        },
        "cul_sum": {
          "cumulative_sum": {
            "buckets_path": "bytes_sum"
          }
        },
        "bucket_script": {
          "cumulative_sum": {
            "buckets_path": "bytes_sum"
          }
      }
    }
  }
}